# model conversion: onnx to tflite

In [ ]:
from pathlib import Path 
import torchaudio as ta 
import os 
import pandas as pd 
import numpy as numpy 
from sklearn.model_selection import train_test_split
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
import torchaudio as ta 
import torch 

model_path = "/home/akinwilson/Code/pytorch/output/model/ResNet/model.onnx"
WORKING_DIR = str( Path().cwd() )

In [ ]:
from wwv.Architecture.ResNet.model import ResNet
import torch 
import torch.nn.functional as F 
# from wwv.architecture import Architecture
from wwv.eval import Metric
from wwv.meta import params
from wwv.util import OnnxExporter
import statistics
from wwv.data import AudioDataModule
import wwv.config as cfg
# from wwv.config import Config, FittingCfg, DataPathCfg, ResNetCfg
from wwv.meta import params as params 
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation



torch.cuda.is_available()
cfg_fitting = cfg.Fitting()
cfg_resnet = cfg.ResNet()

cfg = cfg.Config(params)
model = ResNet(num_blocks=cfg_resnet.num_blocks, cfg=cfg)

OnnxExporter(model, cfg, os.getcwd(), 12)()


In [12]:
import sys
import os
import onnx
from onnx_tf.backend import prepare


model_path = "/home/akinwilson/Code/pytorch/notebooks/model.onnx"
out_path = "/home/akinwilson/Code/pytorch/notebooks/tf"
out_lite_path = "/home/akinwilson/Code/pytorch/notebooks/model.tflite"
# Load the ONNX model
# onnx_model = onnx.load(model_path)
# tf_rep = prepare(onnx_model)
# # Get the input tensor shape
# input_tensor = tf_rep.signatures[tf_rep.inputs[0]]
# input_shape = input_tensor.shape
# input_shape
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load(model_path)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(out_path)  # export the model

Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.
Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.
Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.
Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.
Assets written to: /home/akinwilson/Code/pytorch/notebooks/tf/assets
Assets written to: /home/akinwilson/Code/pytorch/notebooks/tf/assets
Assets written to: /home/akinwilson/Code/pytorch/notebooks/tf/assets
Assets written to: /home/akinwilson/Code/pytorch/notebooks/tf/assets


In [15]:
input_names = ['input_mfcc', 'dummy_input']         # the model's input names
output_names = ['output_wwp']                       # the model's output names
dynamic_axes={'input_mfcc' : {0 : 'batch_size'}}


import tensorflow as tf
# make a converter object from the saved tensorflow file
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(out_path + "/saved_model.pb", #TensorFlow freezegraph .pb model file
                                                                input_arrays=input_names, # name of input arrays as defined in torch.onnx.export function before.
                                                                output_arrays=output_names)  # name of output arrays defined in torch.onnx.export function before.
                                                      
# tell converter which type of optimization techniques to use
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional
# convert the model 
tf_lite_model = converter.convert()
# save the converted model 
with open(out_lite_path, 'wb') as file:
    file.write(tf_lite_model)


########################################################################################################
model = trainer.model.module.module.model
from tensorflow import lite
import tensorflow as tf
import tensorflow_model_optimization as tfmot

converter = lite.TFLiteConverter.from_keras_model(model)
                    converter.optimizations = [tf.lite.Optimize.DEFAULT]
                    tflite_model = converter.convert()
                    open(join(model_folder, f'{model_name}_epoch_{result["epoch"]}.tflite'), "wb").write(tflite_model)
                    # Quantized TFLite Model
                    if output_quant:
                        converter = tf.lite.TFLiteConverter.from_keras_model(model)
                        converter.optimizations = [tf.lite.Optimize.DEFAULT]
                        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
                        converter.inference_input_type = tf.float32
                        converter.inference_output_type = tf.float32
                        converter.representative_dataset = self.__representative_dataset__
                        tflite_quant_model = converter.convert()
                        open(join(model_folder, f'{model_name}_epoch_{result["epoch"]}_quant.tflite'), "wb").write(tflite_quant_model)

TypeError: TFLiteConverter.from_frozen_graph() got an unexpected keyword argument 'signature_keys'